# Initialize a timeperiod

This function will go through the provided time period and necessary files to run GEODYN:
  - external attitude
  - PCE (if not constructed)
  - initial conditions (raw from PCE)
  - 

In [1]:
%load_ext autoreload
%autoreload 2

from pygeodyn.PYGEODYN import Pygeodyn
from gc import collect as gc_collect

# ## Initialization timeperiod
### Satellites:
Sat_List = ['spire083',
            #,'spire084',
            #'spire085',
           ]

## Stage 1- EXAT and PCE/Raw ICs

In [2]:
%load_ext autoreload
%autoreload 2

from pygeodyn.PYGEODYN import Pygeodyn
from gc import collect as gc_collect

startdate = "2018-11-08"
enddate   = "2018-11-13"  #24

for sat in Sat_List:

    settings_SPIRE_pce= {# Basic input settings
                 'satellite'      : {'input': sat},
                 'den_model'      : {'input': 'jb2008'},
                 'run_type'       : {'input': 'DataReduction_PCE'},
                 'run_specifier'  : {'input': '_test_infrastruc'},
                 'cd_model'       : {'input': 'BWDRAG'},
                 'file_string'    : {'input': 'CD_2p3'},
               # Force Model settings
                  'cd_type'               : {'input':"Fixed_CD"},
                  'cd_value'              : {'input':2.300000},
                  'scaling_factor'        : {'input':False},
                  'cd_adjustment_boolean' : {'input':False },
                  'hours_between_cd_adj'  : {'input':6 },
               # Run
                  'step'           : {'input': 60.},
                  'orbfil_step'    : {'input': 120.},    
                  #
                  'arc'    : {'input':["2018.324"]},
                  'epoch_start'    : {'input':["2018-11-20 00:00:00"]},
                  'epoch_stop'     : {'input':["2018-11-21 00:00:00"]},
                  #                  
                  'global_options': {'input':'pso_2018'},
               # Request read on raw outputs
                  'request_data'   : {'input': ['Trajectory_orbfil',
                                                'Density',
                                                'DragFile',
                                                'Residuals_summary',
                                               ]},
              #end dict
              }




    sat = Pygeodyn(settings_SPIRE_pce, use_file=False)
    sat.initialize_timeperiod_stage1(startdate, enddate,
                                 overwrite_exat=True, 
                                 overwrite_g2b=True)
    sat = 0
    
    

import sys
sys.exit(0)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
spire083
----------------------------------------------------------------------------
Initializing the time period from 2018-11-08 00:00:00 to 2018-11-14 00:00:00
     overwriting the epoch start and stop to match
----------------------------------------------------------------------------

spire083
Step 0: Make directory structure for satellite input data
Directory Exists:  /data/SatDragModelValidation/data/inputs/sat_spire083
Directory Exists:  /data/SatDragModelValidation/data/inputs/sat_spire083/setups
Directory Exists:  /data/SatDragModelValidation/data/inputs/sat_spire083/external_attitude
Directory Exists:  /data/SatDragModelValidation/data/inputs/sat_spire083/g2b
Step 1: Construct daily external Attitude files
   Making an external attitude file: EXAT01.2018.312
        - converting dates from GPS to TDT.


TypeError: Addition/subtraction of integers and integer-arrays with Timestamp is no longer supported.  Instead of adding/subtracting `n`, use `n * obj.freq`

## Stage 2: Setup files + Raw ICs and Update ICs

At this stage, the external attitude and G2B files for the time period of interest should be constructed for each satellite.

Check that the g2b and raw initial conditions are named according the the constructed files.

In [ ]:
from  datetime import datetime, timedelta
import pandas as pd
import numpy  as np
import linecache


### Find earliest daily start time to match data

Use the earliest instance of data in a day as the starting epoch/IC and set the end epoch to midnight

In [ ]:
def get_dates(test_timeperiod, file_raw_ICs):

    start_0ut = []
    end___0ut = []
    arcs = []
    start_update = []
    #
    dt_1days = pd.to_timedelta(24, 'h') 
    #


    startdate = pd.to_datetime(test_timeperiod[0])
    enddate   = pd.to_datetime(test_timeperiod[-1])
    startdate_dt = pd.to_datetime(startdate, format='%Y-%m-%d')
    enddate_dt   = pd.to_datetime(enddate,   format='%Y-%m-%d')
    starts_linspace_dt = pd.date_range(start=startdate_dt ,
                                         end=enddate_dt   ,
                                        freq=str(1)+"D")

    ### Make a list of the initial conditions for daily epoch start times that match
    ### the first instance of pce data for that day
    for iday, day in enumerate(starts_linspace_dt):

        epoch_start =  f"{day.strftime('%Y-%m-%d %H:%M:%S')}" #f"2018-11-{dayval:02d} 00:00:00"
        print(epoch_start)
        epoch_startDT = pd.to_datetime(epoch_start,format='%Y-%m-%d %H:%M:%S')



        datetype = 'datetime_string'
        date_in_file_flag = False

        # Only need to use accuracy to within 1 second (ignore the microseconds in the file)

        if datetype == 'datetime_string':
            date_str = str(epoch_startDT)
        elif datetype == 'YYMMDDHHMMSS':
            date_str = datetime.strftime(epoch_startDT, '%y%m%d%H%M%S')

        with open(file_raw_ICs, 'r') as f:
            for line_no, line_text in enumerate(f):
                if date_str in line_text:
                    date_in_file_flag = True
                    #                 print('    ','xyzline',line_no,line_text)
                    break
        if date_in_file_flag == False:
            #         print(date_str,'not found in file.')

            # Find the dates that have the same hour
            if datetype == 'datetime_string':
                date_roundhour_str = str(epoch_startDT)[:10]
            elif datetype == 'YYMMDDHHMMSS':
                date_roundhour_str = datetime.strftime(epoch_startDT, '%y%m%d')
#             print("date_roundhour_str", date_roundhour_str)
            # Scan through IC file and append a list of dates within the same hour
            line_no_list = []
            line_list = []
            with open(file_raw_ICs, 'r') as f:
                for line_no, line_text in enumerate(f):
                    if date_roundhour_str in line_text:
                        line_no_list.append(line_no)
            
            
            ### If data does not contains this day, don't append it
            if len(line_no_list) ==0:
                print("** no data for ", date_roundhour_str)
                continue
                
                
            # print(line_no_list)
            for i in np.arange(line_no_list[2], line_no_list[-1]):
                line = linecache.getline(file_raw_ICs, i)
                line_list.append(line)
            dates = []
            for i, val in enumerate(line_list):
                if datetype == 'datetime_string':
                    dates.append(
                        pd.to_datetime(line_list[i][:19],
                                       format='%Y-%m-%d %H:%M:%S'))
                elif datetype == 'YYMMDDHHMMSS':
                    dates.append(
                        pd.to_datetime(line_list[i][:19],
                                       format='%y%m%d%H%M%S.%f'))

            start_update.append(dates[1])

            
        
        else:
            #         print('Found date in IC file:', str(epoch_startDT))
            xyzline = pd.read_csv(
                file_raw_ICs,
                skiprows=line_no,
                nrows=1,
                sep='\s+',
                dtype=str,
                names=[
                    'DateYMD',
                    'DateHMS',
                    'X',
                    'Y',
                    'Z',
                    'X_dot',
                    'Y_dot',
                    'Z_dot',
                ],
            )
            start_update.append(
                pd.to_datetime(xyzline['DateYMD'] + xyzline['DateHMS'],
                               format='%Y-%m-%d%H:%M:%S')[0])

#         print(epoch_start[:10], "earliest time is:",
#               start_update[iday])

        print("---appending dates for ", day)
        start_0ut.append(f"{day.strftime('%Y-%m-%d')} 00:00:00")#f"2018-11-{dayval:02d} 00:00:00")
        end___0ut.append(
            pd.to_datetime(epoch_startDT +
                           dt_1days).strftime('%Y-%m-%d %H:%M:%S'))
        arcs.append(epoch_startDT.strftime('%Y.%j'))

#     del date_roundhour_str
#     del line_no_list
#     del line_list
#     del dates
#     del xyzline

    start_update = [datetime.strftime(idate, '%Y-%m-%d %H:%M:%S') for idate in start_update]
    
    return(start_0ut,end___0ut,arcs,start_update)

In [ ]:
# Sat_List = ['spire083', 'spire084', 'spire085']  
Sat_List = ['spire083']  
# test_timeperiod = ["2018-11-02 00:00:00", "2018-11-29 00:00:00"]  # 4 days
test_timeperiod = ["2018-11-07 00:00:00", "2018-11-08 00:00:00"]  # 4 days


In [ ]:
%load_ext autoreload
%autoreload 2
from pygeodyn.PYGEODYN import Pygeodyn
from gc import collect as gc_collect


obj = {}

for isat,satval in enumerate(Sat_List):
    satnum = int(satval[5:])

    file_raw_ICs = f"/data/SatDragModelValidation/data/inputs/"\
              +f"sat_spire{satnum:03d}/g2b/Spire{satnum:03d}_RawEphem_20181101_20181130.txt"

    start_0ut, end___0ut, arcs, start_update = get_dates(test_timeperiod, file_raw_ICs)

    ### Run GEODYN with earliest start time to midnight
    ###      Runnning GEODYN in this configuration, with the rawephem file as the IC 
    print()
    print("start_0ut",start_0ut )
    print("end___0ut",end___0ut )
    print("arcs",arcs )
    print("start_update",start_update )
    print()


    settings_SPIRE= {# Basic input settings
                     'satellite'      : {'input': f'spire{satnum:03d}'},
                     'den_model'      : {'input': 'jb2008'},
                     'run_type'       : {'input': 'DataReduction_PCE'},
                     'run_specifier'  : {'input': '_updateICs_1'},
                     'cd_model'       : {'input': 'BWDRAG'},
                     'file_string'    : {'input': 'CD_2p3'},
                     # Force Model settings
                      'cd_type'               : {'input':"Fixed_CD"},
                      'cd_value'              : {'input':2.300000},
                      'scaling_factor'        : {'input':False},
                      'cd_adjustment_boolean' : {'input':False },
                      'hours_between_cd_adj'  : {'input':6 },
                     # Run
                      'step'           : {'input': 60.},
                      'orbfil_step'    : {'input': 120.},
                      'which_ICfile'   : {'input':file_raw_ICs},
                       #
                      'arc'            : {'input':arcs},
                      'epoch_start'    : {'input':start_update},
                       #
                      'epoch_stop'     : {'input':end___0ut},       
                       #
                      'global_options' : {'input':'pso_2018'},
                     # Request read on raw outputs
                      'request_data'   : {'input': ['Trajectory_orbfil', 
                                                   'Density', 
                                                   'Residuals_summary',
                                                   'RunSummary',
                                                   'DragFile',
                                                   ]},
                  #end dict
                  }
    sat = Pygeodyn(settings_SPIRE, use_file=False)
    sat.run_arcs()
    obj[satval] = sat.getData()
#     obj[satval] =  sat.getData_BigData_lowmemory()

In [ ]:
obj[satval].__dict__.keys()

In [ ]:
obj[satval].__dict__['global_params']

In [ ]:
obj[satval].__dict__['run_parameters2018.311']

In [ ]:
# obj[satval].__dict__['OrbitResids']['2018.313.01'].keys()
# obj[satval].__dict__['OrbitResids']['2018.313.01']['data_PCE']
# obj[satval].__dict__['OrbitResids']['2018.313.01']['data_orbfil']

# obj[satval].__dict__.keys()

# obj[satval].__dict__["Statistics"]

In [ ]:
# import plotly.graph_objects as go
# from plotly.offline import plot, iplot
# from plotly.subplots import make_subplots
# import plotly.express as px

# config = dict({
#                 'displayModeBar': True,
#                 'responsive': False,
#                 'staticPlot': True,
#                 'displaylogo': False,
#                 'showTips': False,
#                 })


# fig = make_subplots(rows=3, cols=1)


# arc = '2018.313.01'

# orbfit = obj[satval].__dict__['OrbitResids'][arc]['data_orbfil']
# pce = obj[satval].__dict__['OrbitResids'][arc]['data_PCE']

# fig.add_trace(go.Scattergl(
#             x=pd.to_datetime(pce['Date']),
#             y=pce['N'],
#                 mode='markers',
#                 opacity=1,
#                 marker=dict(size=3, color="blue"),
#                 showlegend=False),
#                 secondary_y=False, row=1, col=1,)

# fig.add_trace(go.Scattergl(
#             x=pd.to_datetime(pce['Date']),
#             y=pce['T'],
#                 mode='markers',
#                 opacity=1,
#                 marker=dict(size=3, color="blue"),
#                 showlegend=False),
#                 secondary_y=False, row=2, col=1,)

# fig.add_trace(go.Scattergl(
#             x=pd.to_datetime(pce['Date']),
#             y=pce['W'],
#                 mode='markers',
#                 opacity=1,
#                 marker=dict(size=3, color="blue"),     
#                 showlegend=False),
#                 secondary_y=False, row=3, col=1,)

# ### ==============================================================


# fig.add_trace(go.Scattergl(
#             x=pd.to_datetime(orbfit['Date']),
#             y=orbfit['N'],
#                 mode='markers',
#                 opacity=1,
#                 marker=dict(size=3, color='red'),
#                 showlegend=False),
#                 secondary_y=False, row=1, col=1,)

# fig.add_trace(go.Scattergl(
#             x=pd.to_datetime(orbfit['Date']),
#             y=orbfit['T'],
#                 mode='markers',
#                 opacity=1,
#                 marker=dict(size=3, color='red'),
#                 showlegend=False),
#                 secondary_y=False, row=2, col=1,)

# fig.add_trace(go.Scattergl(
#             x=pd.to_datetime(orbfit['Date']),
#             y=orbfit['W'],
#                 mode='markers',
#                 opacity=1,
#                 marker=dict(size=3, color='red'),     
#                 showlegend=False),
#                 secondary_y=False, row=3, col=1,)

# fig.update_yaxes( title="N",  exponentformat= 'power',row=1, col=1)
# fig.update_yaxes( title="T",  exponentformat= 'power',row=2, col=1)
# fig.update_yaxes( title="W",  exponentformat= 'power',row=3, col=1)


# fig.update_layout(
#         autosize=True
# #         width=800,height=900,
#                 )
# fig.show(config=config)


In [ ]:
arc = '2018.311'
obj[satval].__dict__.keys()

In [ ]:
# obj[satval].__dict__["Trajectory_orbfil"][arc]['data_record']

In [ ]:
StateVector_PCE_datafile =    f"/data/SatDragModelValidation/data/inputs/"\
                              +f"sat_spire{satnum:03d}/g2b/Spire{satnum:03d}_RawEphem_20181101_20181130.txt"


# self.__dict__['global_params']['file_statevector_ICs']
StateVector_PCE_datafile
arc_first_time  = obj[satval].__dict__['Trajectory_orbfil'][arc]['data_record']['Date_UTC'].iloc[0]
arc_last_time   = obj[satval].__dict__['Trajectory_orbfil'][arc]['data_record']['Date_UTC'].iloc[-1]

arc_first_time_str     =  str(arc_first_time)#.replace( "'",' ') 
arc_last_time_str      =  str(arc_last_time)#.replace( "'",' ') 


A=[]
for i,val in enumerate(np.arange(-20,20)):
    A.append(str(pd.to_datetime(arc_first_time)+pd.to_timedelta(val,'s')))
B=[]
for i,val in enumerate(np.arange(-20,20)):
    B.append(str(pd.to_datetime(arc_last_time)+pd.to_timedelta(val,'s')))

####---------------------------------------------------------
last_line = False
get_firstline = True
with open(StateVector_PCE_datafile, 'r') as f:
    for line_no, line_text in enumerate(f):
        if any(times in line_text for times in A) and get_firstline==True:
            first_line = line_no
            get_firstline = False
        if any(times in line_text for times in B):
            last_line = line_no
            break

    if not last_line:
        last_line = first_line +32220
        print('No matching lastline time: ',arc_last_time_str, last_line )

print("first_line", first_line)
print("last_line" , last_line)
print("arc_first_time", arc_first_time)
print("arc_last_time", arc_last_time)


In [ ]:
PCE_data = pd.read_csv(StateVector_PCE_datafile, 
            skiprows = first_line, 
#             nrows=last_line - first_line,           
            sep = '\s+',
            dtype=object,
            names = [
                'DateYMD',
                'DateHMS',
                'X_pce',
                'Y_pce',
                'Z_pce',
                'Xdot_pce',
                'Ydot_pce',
                'Zdot_pce',
    ],)

PCE_data['Date_pd'] =  pd.to_datetime(PCE_data['DateYMD']+PCE_data['DateHMS'], format='%Y-%m-%d%H:%M:%S')
del PCE_data['DateYMD'], PCE_data['DateHMS']


In [ ]:
PCE_data = PCE_data.query(f"{arc_first_time.year}{arc_first_time.month:02d}{arc_first_time.day:02d}"\
                     +f" < Date_pd < "\
                     +f"{arc_last_time.year}{arc_last_time.month:02d}{arc_last_time.day+1:02d}")


In [ ]:
# PCE_data

In [ ]:
# obj[satval].__dict__["Trajectory_orbfil"][arc]['data_record']

In [ ]:
CombinedOrbitsDF  = {}


orbfil_arc1 = obj[satval].__dict__['Trajectory_orbfil'][arc]['data_record']
orbfil_arc1['Date_pd'] = pd.to_datetime(orbfil_arc1['Date_UTC'])

# del orbfil_arc1['Date_UTC']
# del orbfil_arc1['MJDSEC ET']
# del orbfil_arc1['Geodetic Latitude']
# del orbfil_arc1['East Longitude']
# del orbfil_arc1['Height']
# del orbfil_arc1['MJDS_UTC']


### CombinedOrbitsDF is a dataframe containing all data between the two files where the dates match
CombinedOrbitsDF[arc] = pd.merge(\
                            left=orbfil_arc1, left_on='Date_pd',
                            right=PCE_data, right_on='Date_pd')

#             print(CombinedOrbitsDF[arc].columns)
CombinedOrbitsDF[arc] = CombinedOrbitsDF[\
                                        arc].rename(\
                                            columns={\
                                        "X j2000" : "X_orbfil"   ,
                                        "Y j2000" : "Y_orbfil"   ,
                                        "Z j2000" : "Z_orbfil"   ,
                                    "X_dot j2000" : "Xdot_orbfil",
                                    "Y_dot j2000" : "Ydot_orbfil",
                                    "Z_dot j2000" : "Zdot_orbfil",
                                                                })


In [ ]:
from pygeodyn.util_dir.coordinate_systems import Convert_cartesian_to_NTW_returnall
rsw_bool= False

In [ ]:
OrbitResids = {} 

############--------------------------------------------------------------------------------------------------        
#             OrbitResids = self.ResidInvestigation_get_residuals_coordsystems(CombinedOrbitsDF)

data_orbfil = {}
data_PCE    = {}
resids      = {}
### Convert the PCE data to NTW
#             print('        Converting PCE data to other coordinates...')
X = CombinedOrbitsDF[arc]['X_pce'].astype(float)
Y = CombinedOrbitsDF[arc]['Y_pce'].astype(float)
Z = CombinedOrbitsDF[arc]['Z_pce'].astype(float)
Xdot = CombinedOrbitsDF[arc]['Xdot_pce'].astype(float)
Ydot = CombinedOrbitsDF[arc]['Ydot_pce'].astype(float)
Zdot = CombinedOrbitsDF[arc]['Zdot_pce'].astype(float)
state_vector = np.transpose(np.array([CombinedOrbitsDF[arc]['X_pce'].astype(float),
                                      Y,
                                      Z,
                                      Xdot,
                                      Ydot,
                                      Zdot]))
data_PCE['Date'] = CombinedOrbitsDF[arc]['Date_pd']

##### NTW Coordinate System
#             NTW_PCE  = [Convert_cartesian_to_NTW_returnall(x) for x in state_vector]
NTW_pce =  [Convert_cartesian_to_NTW_returnall(x_pce, 0, True) \
                        for x_pce in state_vector]
Tmat_ntw         = []
n_pce        = []
t_pce        = []
w_pce        = []
for vec,matrix in NTW_pce:
    Tmat_ntw.append(matrix)
    n_pce.append( vec[0])
    t_pce.append( vec[1])
    w_pce.append( vec[2])        

data_PCE['N'] = n_pce
data_PCE['T'] = t_pce
data_PCE['W'] = w_pce
##### XYZ Coordinate System
data_PCE['X'] = X
data_PCE['Y'] = Y
data_PCE['Z'] = Z
data_PCE['Xdot'] = Xdot
data_PCE['Ydot'] = Ydot
data_PCE['Zdot'] = Zdot


### Convert the ORBIT FILE data to NTW
X = CombinedOrbitsDF[arc]['X_orbfil']
Y = CombinedOrbitsDF[arc]['Y_orbfil']
Z = CombinedOrbitsDF[arc]['Z_orbfil']
Xdot = CombinedOrbitsDF[arc]['Xdot_orbfil']
Ydot = CombinedOrbitsDF[arc]['Ydot_orbfil']
Zdot = CombinedOrbitsDF[arc]['Zdot_orbfil']
state_vector = np.transpose(np.array([X, Y, Z, Xdot, Ydot, Zdot]))
data_orbfil['Date'] = CombinedOrbitsDF[arc]['Date_pd']

##### NTW Coordinate System
#             NTW_orbfil  = [Convert_cartesian_to_NTW_returnall(x) for x in state_vector]
NTW_orb  = [Convert_cartesian_to_NTW_returnall(x_orb, Tmat_ntw_i, False) \
                    for x_orb, Tmat_ntw_i in zip(state_vector,Tmat_ntw)]
n_orb        = []
t_orb        = []
w_orb        = []

for vecorb,matrixorb in NTW_orb:
    n_orb.append( vecorb[0])
    t_orb.append( vecorb[1])
    w_orb.append( vecorb[2])        

data_orbfil['N'] = n_orb
data_orbfil['T'] = t_orb
data_orbfil['W'] = w_orb
##### XYZ Coordinate System
data_orbfil['X'] = X
data_orbfil['Y'] = Y
data_orbfil['Z'] = Z
data_orbfil['Xdot'] = Xdot
data_orbfil['Ydot'] = Ydot
data_orbfil['Zdot'] = Zdot


##### R theta phi Coordinate System
#             data_orbfil['R']     = np.sqrt( np.square(X) + 
#                                             np.square(Y) +
#                                             np.square(Z) )
#             data_orbfil['theta'] = np.arctan(Y / X)
#             data_orbfil['phi']   = np.arccos(Z / (np.sqrt( np.square(X) + 
#                                             np.square(Y) +
#                                             np.square(Z) )))


### RESIDUALS:
resids['Date'] = CombinedOrbitsDF[arc]['Date_pd']

##### NTW Coordinate System
resids['N'] = (np.array(data_PCE['N']) - np.array(data_orbfil['N']))
resids['T'] = (np.array(data_PCE['T']) - np.array(data_orbfil['T']))
resids['W'] = (np.array(data_PCE['W']) - np.array(data_orbfil['W']))


OrbitResids[arc] = {}
OrbitResids[arc]['data_orbfil'] = data_orbfil
OrbitResids[arc]['data_PCE']    = data_PCE
OrbitResids[arc]['resids']      = resids


In [ ]:
import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px

config = dict({
                'displayModeBar': True,
                'responsive': True,
                'staticPlot': False,
                'displaylogo': False,
                'showTips': False,
                })


fig = make_subplots(rows=3, cols=1)


arc = '2018.311.01'


orbfit = data_orbfil
pce =data_PCE

fig.add_trace(go.Scattergl(
            x=pd.to_datetime(pce['Date']),
            y=pce['N'],
            name=f"PCE, Spire {satnum}",
                mode='markers',
                opacity=1,
                marker=dict(size=3, color="blue"),
                showlegend=True),
                secondary_y=False, row=1, col=1,)

fig.add_trace(go.Scattergl(
            x=pd.to_datetime(pce['Date']),
            y=pce['T'],
                mode='markers',
                opacity=1,
                marker=dict(size=3, color="blue"),
                showlegend=False),
                secondary_y=False, row=2, col=1,)

fig.add_trace(go.Scattergl(
            x=pd.to_datetime(pce['Date']),
            y=pce['W'],
                mode='markers',
                opacity=1,
                marker=dict(size=3, color="blue"),     
                showlegend=False),
                secondary_y=False, row=3, col=1,)

### ==============================================================


fig.add_trace(go.Scattergl(
            x=pd.to_datetime(orbfit['Date']),
            y=orbfit['N'],
                name=f"Orbit Fit, Spire {satnum}",
                mode='markers',
                opacity=1,
                marker=dict(size=3, color='red'),
                showlegend=True),
                secondary_y=False, row=1, col=1,)

fig.add_trace(go.Scattergl(
            x=pd.to_datetime(orbfit['Date']),
            y=orbfit['T'],
                mode='markers',
                opacity=1,
                marker=dict(size=3, color='red'),
                showlegend=False),
                secondary_y=False, row=2, col=1,)

fig.add_trace(go.Scattergl(
            x=pd.to_datetime(orbfit['Date']),
            y=orbfit['W'],
                mode='markers',
                opacity=1,
                marker=dict(size=3, color='red'),     
                showlegend=False),
                secondary_y=False, row=3, col=1,)

fig.update_yaxes( title="N",  exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="T",  exponentformat= 'power',row=2, col=1)
fig.update_yaxes( title="W",  exponentformat= 'power',row=3, col=1)


# fig.update_layout(f"Spire {satnum} NTW, direct comparison",
#         autosize=True,
#         width=800,height=900,
#                 )
fig.show(config=config)


In [ ]:


fig = make_subplots(rows=3, cols=1)


# arc = '2018.313.01'

# orbfit = obj[satval].__dict__['OrbitResids'][arc]['data_orbfil']
# pce = obj[satval].__dict__['OrbitResids'][arc]['data_PCE']

fig.add_trace(go.Scattergl(
            x=resids['Date'],
            y=resids['N'],
                mode='markers',
                opacity=1,
                marker=dict(size=3, color="blue"),
                showlegend=False),
                secondary_y=False, row=1, col=1,)

fig.add_trace(go.Scattergl(
            x=resids['Date'],
            y=resids['T'],
                mode='markers',
                opacity=1,
                marker=dict(size=3, color="blue"),
                showlegend=False),
                secondary_y=False, row=2, col=1,)

fig.add_trace(go.Scattergl(
            x=resids['Date'],
            y=resids['W'],
                mode='markers',
                opacity=1,
                marker=dict(size=3, color="blue"),     
                showlegend=False),
                secondary_y=False, row=3, col=1,)

### ==============================================================


# fig.add_trace(go.Scattergl(
#             x=CombinedOrbitsDF[arc]['Date_pd'],
#             y=CombinedOrbitsDF[arc]['X_pce'],
#                 mode='markers',
#                 opacity=1,
#                 marker=dict(size=3, color='red'),
#                 showlegend=False),
#                 secondary_y=False, row=1, col=1,)

# fig.add_trace(go.Scattergl(
#             x=CombinedOrbitsDF[arc]['Date_pd'],
#             y=CombinedOrbitsDF[arc]['Y_pce'],
#                 mode='markers',
#                 opacity=1,
#                 marker=dict(size=3, color='red'),
#                 showlegend=False),
#                 secondary_y=False, row=2, col=1,)

# fig.add_trace(go.Scattergl(
#             x=CombinedOrbitsDF[arc]['Date_pd'],
#             y=CombinedOrbitsDF[arc]['Z_pce'],
#                 mode='markers',
#                 opacity=1,
#                 marker=dict(size=3, color='red'),     
#                 showlegend=False),
#                 secondary_y=False, row=3, col=1,)

fig.update_yaxes( title="N-residual (m)",  exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="T-residual (m)",  exponentformat= 'power',row=2, col=1)
fig.update_yaxes( title="W-residual (m)",  exponentformat= 'power',row=3, col=1)


fig.update_layout(title=f"Spire {satnum} Residuals for PCE-Orbfil, NTW",
                  
        autosize=True
#         width=800,height=900,
                )
fig.show(config=config)


In [ ]:

fig = make_subplots(rows=3, cols=1)


arc = '2018.311'

# orbfit = obj[satval].__dict__['OrbitResids'][arc]['data_orbfil']
# pce = obj[satval].__dict__['OrbitResids'][arc]['data_PCE']
CombinedOrbitsDF[arc]

fig.add_trace(go.Scattergl(
            x=CombinedOrbitsDF[arc]['Date_pd'],
            y=CombinedOrbitsDF[arc]['X_pce'].values.astype(float) - CombinedOrbitsDF[arc]['X_orbfil'],
                mode='markers',
                opacity=1,
                marker=dict(size=3, color="blue"),
                showlegend=False),
                secondary_y=False, row=1, col=1,)

fig.add_trace(go.Scattergl(
            x=CombinedOrbitsDF[arc]['Date_pd'],
            y=CombinedOrbitsDF[arc]['Y_pce'].values.astype(float) - CombinedOrbitsDF[arc]['Y_orbfil'],
                mode='markers',
                opacity=1,
                marker=dict(size=3, color="blue"),
                showlegend=False),
                secondary_y=False, row=2, col=1,)

fig.add_trace(go.Scattergl(
            x=CombinedOrbitsDF[arc]['Date_pd'],
            y=CombinedOrbitsDF[arc]['Z_pce'].values.astype(float) - CombinedOrbitsDF[arc]['Z_orbfil'],
                mode='markers',
                opacity=1,
                marker=dict(size=3, color="blue"),     
                showlegend=False),
                secondary_y=False, row=3, col=1,)

### ==============================================================


# fig.add_trace(go.Scattergl(
#             x=CombinedOrbitsDF[arc]['Date_pd'],
#             y=CombinedOrbitsDF[arc]['X_pce'],
#                 mode='markers',
#                 opacity=1,
#                 marker=dict(size=3, color='red'),
#                 showlegend=False),
#                 secondary_y=False, row=1, col=1,)

# fig.add_trace(go.Scattergl(
#             x=CombinedOrbitsDF[arc]['Date_pd'],
#             y=CombinedOrbitsDF[arc]['Y_pce'],
#                 mode='markers',
#                 opacity=1,
#                 marker=dict(size=3, color='red'),
#                 showlegend=False),
#                 secondary_y=False, row=2, col=1,)

# fig.add_trace(go.Scattergl(
#             x=CombinedOrbitsDF[arc]['Date_pd'],
#             y=CombinedOrbitsDF[arc]['Z_pce'],
#                 mode='markers',
#                 opacity=1,
#                 marker=dict(size=3, color='red'),     
#                 showlegend=False),
#                 secondary_y=False, row=3, col=1,)

fig.update_yaxes( title="X",  exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="Y",  exponentformat= 'power',row=2, col=1)
fig.update_yaxes( title="Z",  exponentformat= 'power',row=3, col=1)


fig.update_layout(title='Residuals for PCE-Orbfil, XYZ',
                  
        autosize=True
#         width=800,height=900,
                )
fig.show(config=config)


 Test by converting these to NTW, check the units, check the coordinate systems.......... yikes


In [ ]:


fig = make_subplots(rows=3, cols=1)


# arc = '2018.313.01'

# orbfit = obj[satval].__dict__['OrbitResids'][arc]['data_orbfil']
# pce = obj[satval].__dict__['OrbitResids'][arc]['data_PCE']
CombinedOrbitsDF[arc]

fig.add_trace(go.Scattergl(
            x=CombinedOrbitsDF[arc]['Date_pd'],
            y=CombinedOrbitsDF[arc]['Xdot_pce'].values.astype(float),
                mode='markers',
                opacity=1,
                marker=dict(size=3, color="blue"),
                showlegend=False),
                secondary_y=False, row=1, col=1,)

fig.add_trace(go.Scattergl(
            x=CombinedOrbitsDF[arc]['Date_pd'],
            y=CombinedOrbitsDF[arc]['Ydot_pce'].values.astype(float),
                mode='markers',
                opacity=1,
                marker=dict(size=3, color="blue"),
                showlegend=False),
                secondary_y=False, row=2, col=1,)

fig.add_trace(go.Scattergl(
            x=CombinedOrbitsDF[arc]['Date_pd'],
            y=CombinedOrbitsDF[arc]['Zdot_pce'].values.astype(float),
                mode='markers',
                opacity=1,
                marker=dict(size=3, color="blue"),     
                showlegend=False),
                secondary_y=False, row=3, col=1,)

### ==============================================================


fig.add_trace(go.Scattergl(
            x=CombinedOrbitsDF[arc]['Date_pd'],
            y=CombinedOrbitsDF[arc]['Xdot_orbfil'],
                mode='markers',
                opacity=1,
                marker=dict(size=3, color='red'),
                showlegend=False),
                secondary_y=False, row=1, col=1,)

fig.add_trace(go.Scattergl(
            x=CombinedOrbitsDF[arc]['Date_pd'],
            y=CombinedOrbitsDF[arc]['Ydot_orbfil'],
                mode='markers',
                opacity=1,
                marker=dict(size=3, color='red'),
                showlegend=False),
                secondary_y=False, row=2, col=1,)

fig.add_trace(go.Scattergl(
            x=CombinedOrbitsDF[arc]['Date_pd'],
            y=CombinedOrbitsDF[arc]['Zdot_orbfil'],
                mode='markers',
                opacity=1,
                marker=dict(size=3, color='red'),     
                showlegend=False),
                secondary_y=False, row=3, col=1,)

fig.update_yaxes( title="Xdot",  exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="Ydot",  exponentformat= 'power',row=2, col=1)
fig.update_yaxes( title="Zdot",  exponentformat= 'power',row=3, col=1)


fig.update_layout(title='PCE vs Orbfil, VEL-XYZ',
                  
        autosize=True
#         width=800,height=900,
                )
fig.show(config=config)


In [ ]:
import sys
sys.exit(0)

In [ ]:
#     obj = sat.getData()



#     print()
#     ### Load the data, grab the final trajectory point from the previous day, use this value as initial condition.
#     IC_update_0ut=[]
#     for iarc, valarc in enumerate(obj.__dict__['global_params']['arc_input']):
#         print('Writing values for', valarc)
#         IC_update_0ut.append([\
#                 obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['X j2000'].values[-1],
#                 obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Y j2000'].values[-1],
#                 obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Z j2000'].values[-1],
#                 obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['X_dot j2000'].values[-1],
#                 obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Y_dot j2000'].values[-1],
#                 obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Z_dot j2000'].values[-1]
#                             ])
#     print(IC_update_0ut)







#     ### Run with 0ut to 0ut
#     ###    Run for the standard full day using the state vector 
#     ###    values from midnight of the previous day as the epoch
#     ###    start initial conditions.  This method truncates the 
#     ###    first day in the original epoch list.

#     settings_SPIRE= {# Basic input settings
#                      'satellite'      : {'input': f'spire{satnum:03d}'},
#                      'den_model'      : {'input': 'jb2008'},
#                      'run_type'       : {'input': 'DataReduction_PCE'},
#                      'run_specifier'  : {'input': '_updateICs_2'},
#                      'cd_model'       : {'input': 'BWDRAG'},
#                      'file_string'    : {'input': 'CD_2p3'},
#                    # Force Model settings
#                       'cd_type'               : {'input':"Fixed_CD"},
#                       'cd_value'              : {'input':2.300000},
#                       'scaling_factor'        : {'input':False},
#                       'cd_adjustment_boolean' : {'input':False },
#                       'hours_between_cd_adj'  : {'input':6 },
#                    # Run
#                       'step'           : {'input': 60.},
#                       'orbfil_step'    : {'input': 60.},    

#                       #
#                       'arc'            : {'input':arcs[1:]},
#                       'epoch_start'    : {'input':start_0ut[1:]},
#                       'epoch_stop'     : {'input':end___0ut[1:]},       
#                       'initial_conditions':{'input':IC_update_0ut[:-1]},

#                       'global_options' : {'input':'pso_2018'},
#                    # Request read on raw outputs
#                       'request_data'   : {'input': ['Trajectory_orbfil', 
#                                                     'RunSummary']},
#                   #end dict
#                   }
#     sat = Pygeodyn(settings_SPIRE, use_file=False)
#     sat.run_arcs()
#     obj = sat.getData()




#     ### Make updated IC file
#     ### Write to an ascii text file. 
#     file_save = f"/data/SatDragModelValidation/data/inputs/sat_spire{satnum:03d}/setups/" \
#                +f"Spire{satnum:03d}_initialconditions_{start_0ut[1:][0][:10]}_{end___0ut[1:][-1][:10]}.txt"

#     f = open(file_save, "w+")
#     f.write("\n")
#     f.close()
#     #                     filemodels = open("/data/geodyn_proj/pygeodyn/temp_runfiles/geodyn_modelpaths.txt","w+")
#     #                     filemodels.write(self.model_data_path+'\n')
#     #                     filemodels.write(self.orbitcloud_csv_file+  '\n')
#     #                     filemodels.close()


#     arc0 = settings_SPIRE['arc']['input'][0]
#     with open(file_save, 'r+') as file:
#         #### Manually write the header units
#         header_units =\
#                     f"{'UTC'.rjust(len(str(obj.__dict__['Trajectory_orbfil'][arc0]['data_record']['Date_UTC'][1]))-1,' ') }"\
#                 +f"  {'(m)'.rjust(15,' ')}"\
#                 +f"  {'(m)'.rjust(15,' ')}"\
#                 +f"  {'(m)'.rjust(15,' ')}"\
#                 +f"  {'(m/s)'.rjust(15,' ')}"\
#                 +f"  {'(m/s)'.rjust(15,' ')}"\
#                 +f"  {'(m/s)'.rjust(15,' ')}"\

#         #### Manually write the header field names
#         header_names =\
#                     f"{'Date'.rjust(len(str(obj.__dict__['Trajectory_orbfil'][arc0]['data_record']['Date_UTC'][1]))-1,' ') }"\
#                 +f"  {'X'.rjust(15,' ')}"\
#                 +f"  {'Y'.rjust(15,' ')}"\
#                 +f"  {'Z'.rjust(15,' ')}"\
#                 +f"  {'X_dot'.rjust(15,' ')}"\
#                 +f"  {'Y_dot'.rjust(15,' ')}"\
#                 +f"  {'Z_dot'.rjust(15,' ')}"\


#     #---+----1----+----2----+----3----+----4----+----5----+----6----+----7----+----8
#         #### Manually write the detailed header description
#         header_meta = \
#         f'''### Initial conditions file
# ### -----------------------
# ###     Satellite: Spire_{satnum:03d}
# ###     Last modified: {datetime.now()-timedelta(hours=7)}
# ###
# ### Source
# ### -------
# ###     Initial conditions from Converged Solutions of GEODYN data reduction.
# ###     Constructed as follows:
# ###        Stage 1
# ###           - for each day of interest, find the earliest epoch time in that
# ###             day that matches the raw data (PCE)
# ###           - set epoch_start value to be that time for each day, respectively
# ###           - run GEODYN with the earliest time of day until midnight.
# ###           - Collect the trajectory outputs into a file, saving just
# ###             state vector of the starttime and endtime
# ###        Stage 2  
# ###           - Set the epoch start time to midnight
# ###           - Use the final state vector values from the previous day
# ###             (should be within 1 minute of midnight) as the new
# ###             initial condition
# ###           - Run geodyn for each daily arc from 0 UT to 0 UT
# ###        Stage 3 
# ###           - correct any failed convergences
# ###           - use the output orbit solutions from the converged solutions
# ###             as the updated initial conditions
# ###      note-- Orbits were fit to PCE (SpireLeoOrb 'POD' from RTOrb) 
# ###             using Jb2008 density model.
# ###
# ### Contents
# ### --------
# ###     Date: (YYYY-MM-DD hh:mm:ss.ssssss) (UTC)
# ###     pvi: Position and velocity (X, Y, Z, X_dot, Y_dot, Z_dot)
# ###          coordinate: ECI-J2000
# ###          unit: m and m/s
# ###
# #{header_units}
# #{header_names}
# ### %eoh
# '''
#         file.write(header_meta)


#         for iarc, valarc in enumerate(obj.__dict__['global_params']['arc_input']):
#             print('Writing values for', valarc)
#         #     update_0ut_sv.append([\
#         #             obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['X j2000'].values[-1],
#         #             obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Y j2000'].values[-1],
#         #             obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Z j2000'].values[-1],
#         #             obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['X_dot j2000'].values[-1],
#         #             obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Y_dot j2000'].values[-1],
#         #             obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Z_dot j2000'].values[-1]
#         #                         ])
#             orbit_data = obj.__dict__['Trajectory_orbfil'][valarc]['data_record']

#             for ii, val in enumerate(orbit_data['Date_UTC']):
#                 ### Only print the hourly values (when the minute value is 0)
#     #             if orbit_data['Date_UTC'][ii].minute==0:
#                 row =   f"{orbit_data['Date_UTC'][ii]}"         \
#                        +f"  {orbit_data['X j2000'][ii]:15.5f}"    \
#                        +f"  {orbit_data['Y j2000'][ii]:15.5f}"    \
#                        +f"  {orbit_data['Z j2000'][ii]:15.5f}"    \
#                        +f"  {orbit_data['X_dot j2000'][ii]:15.5f}"\
#                        +f"  {orbit_data['Y_dot j2000'][ii]:15.5f}"\
#                        +f"  {orbit_data['Z_dot j2000'][ii]:15.5f}"\
#                        + f"\n"
#             #     print(row)
#                 file.write(row)

